In [1]:
import pandas as pd

df = pd.DataFrame(pd.read_csv('reviews.csv', sep = ',', header = 0, index_col = 0))
df = df.reset_index()
df.head()

,Rating,Year_Month,Reviewer_Location,Review_Text
0,5,2019-3,United Arab Emirates,"We've been to Disneyland Hongkong and Tokyo, s..."
1,4,2018-6,United Kingdom,I went to Disneyland Paris in April 2018 on Ea...
2,5,2019-4,United Kingdom,"What a fantastic place, the queues were decent..."
3,4,2019-4,Australia,We didn't realise it was school holidays when ...
4,5,missing,France,A Trip to Disney makes you all warm and fuzzy ...


In [2]:
# print(df)

# Suppression des 'missing'

# df[df['Year_Month'].str.contains('missing')==True].count().value_counts()
# supp = df[df['Year_Month'].str.contains('missing')==True].index
# print(supp)
# df = df.drop(supp)
# df.shape

df[df['Review_Text'].str.contains('missing')==True].count().value_counts()
# df=df.drop(df[df['Review_Text'].str.contains('missing')==True].index)
# df.shape

# df[df['Year_Month'].str.contains('missing')==True].count().value_counts()
# # df=df.drop(df[df['Review_Text'].str.contains('missing')==True].index)
# df.head()

213    4
dtype: int64

In [10]:
df[df['Review_Text'].str.contains('missing')]

,Rating,Year_Month,Reviewer_Location,Review_Text
91,1,2019-4,United States,As my daughter grew up in southern California ...
99,3,2019-4,United States,With early morning entrance we had very short ...
177,2,2018-6,Australia,I have visited Disneyland Anaheim probably 10 ...
209,3,2019-2,United Kingdom,I was expecting the same 'Magic' in Disneyland...
210,4,2019-2,United Kingdom,Two established theme parks with lots of rides...
...,...,...,...,...
13557,5,missing,United Kingdom,"We have just got back from Disneyland Paris, a..."
13579,4,missing,United Kingdom,People asked if my fiance and I were going to ...
13587,2,missing,United States,I was very disappointed with Disneyland Paris....
13598,3,missing,United Kingdom,My husband and I have just returned from Disne...


In [11]:
df.loc[99, 'Review_Text']

'With early morning entrance we had very short lines (5 minutes max) so we were done by early afternoon. The rides themselves seem shorter. Indiana Jones is a roller coaster (I miss the Jeep) and there is no Splash Mountain. A few other big rides missing too. It looked liked Disneyland but the figures just stand in spots so people can line up and get pictures. I would spend money elsewhere if you have already been to Disneyland in the states. '

In [ ]:

# Traitement des dates

df['year'] = df['Year_Month'].apply(lambda str : str.split('-')[0])
# df['month'] = df['Year_Month'].apply(lambda str : str.split('-')[1])


df.head()


# print(df['Year_Month'].apply(lambda str : str.split('-')))

,Rating,Year_Month,Reviewer_Location,Review_Text,year
0,5,2019-3,United Arab Emirates,"We've been to Disneyland Hongkong and Tokyo, s...",2019
1,4,2018-6,United Kingdom,I went to Disneyland Paris in April 2018 on Ea...,2018
2,5,2019-4,United Kingdom,"What a fantastic place, the queues were decent...",2019
3,4,2019-4,Australia,We didn't realise it was school holidays when ...,2019
4,5,missing,France,A Trip to Disney makes you all warm and fuzzy ...,missing


In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = stopwords.words('english')
# custom_stop_words = ['Disney', 'Disneyland', 'Disneyworld']
nltk.download('omw-1.4')

NameError: name 'nltk' is not defined

In [12]:
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
import re
# nltk.download('punkt')
from nltk.stem.snowball import SnowballStemmer

# data = df['Review_Text'][0]

REMPLACE_SANS_ESPACE = re.compile("[;:!\'?,\"()\[\]]")
REMPLACE_AVEC_ESPACE = re.compile("()|(\-)|(\/)|[.]")
PUR_NOMBRE = re.compile("[0-9]")
        
def preprocess(txt):
    txt = PUR_NOMBRE.sub("", txt) # retire les nombres (comme les années)
    txt.replace('\n', ' ')# Retire les \n (retours chariots)
    txt = REMPLACE_SANS_ESPACE.sub(" ", txt)
    txt = REMPLACE_AVEC_ESPACE.sub("", txt)
    txt = txt.lower()
    return txt

df['data'] = df['Review_Text'].apply(lambda x: preprocess(x))

# print(data)
# ppdata = preprocess(data)
# print("ppdata:", ppdata)


stemmer = SnowballStemmer('english') 
df['data'] = df['data'].apply(lambda w: stemmer.stem(w))



eng_stopwords = set(stopwords.words('english'))
filtre_stopen =  lambda text: [token for token in text if token.lower() not in eng_stopwords]
df['data'] = [' '.join(filtre_stopen(word_tokenize(item))) for item in df['data']]
df.head()



# fd = nltk.FreqDist(stpdata) 
# print(fd.most_common())






,Rating,Year_Month,Reviewer_Location,Review_Text,data
0,5,2019-3,United Arab Emirates,"We've been to Disneyland Hongkong and Tokyo, s...",disneyland hongkong tokyo far one best looking...
1,4,2018-6,United Kingdom,I went to Disneyland Paris in April 2018 on Ea...,went disneyland paris april easter weekend kno...
2,5,2019-4,United Kingdom,"What a fantastic place, the queues were decent...",fantastic place queues decent best time year g...
3,4,2019-4,Australia,We didn't realise it was school holidays when ...,realise school holidays went consequently extr...
4,5,missing,France,A Trip to Disney makes you all warm and fuzzy ...,trip disney makes warm fuzzy actual kid big us...


In [ ]:
# Polarisation des ratings :

def setClassBin(i):
    if i > 3:
        return 1
    else:
        return 0

df['polar'] = [setClassBin(x) for x in df.Rating]


bilan=df.groupby(['Rating','polar'])
bilan.size()

df.head()

,Rating,Year_Month,Reviewer_Location,Review_Text,year,data,polar
0,5,2019-3,United Arab Emirates,"We've been to Disneyland Hongkong and Tokyo, s...",2019,disneyland hongkong tokyo far one best looking...,1
1,4,2018-6,United Kingdom,I went to Disneyland Paris in April 2018 on Ea...,2018,went disneyland paris april easter weekend kno...,1
2,5,2019-4,United Kingdom,"What a fantastic place, the queues were decent...",2019,fantastic place queues decent best time year g...,1
3,4,2019-4,Australia,We didn't realise it was school holidays when ...,2019,realise school holidays went consequently extr...,1
4,5,missing,France,A Trip to Disney makes you all warm and fuzzy ...,missing,trip disney makes warm fuzzy actual kid big us...,1


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split 


cv = CountVectorizer(binary=True)
cv.fit(df["data"])





CountVectorizer(binary=True)

In [ ]:
X = df['data']

y = df['polar']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,stratify=y)


In [ ]:
Xf_onehot = cv.transform(df['data'])
pd.DataFrame(Xf_onehot.toarray())

KernelInterrupted: Execution interrupted by the Jupyter kernel.